In [ ]:
import librosa
import matplotlib.pyplot as plt                                                 
import scipy.io.wavfile
import numpy as np
import math
from scipy.fftpack import dct
from silero_vad import load_silero_vad, read_audio, get_speech_timestamps
import cv2
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense
from tensorflow.keras.utils import image_dataset_from_directory
import torchaudio

In [ ]:
def remake_audio(path):
    audio,sampleRate = torchaudio.load(path)
    audio=audio.numpy()
    audio=np.mean(audio,axis=0)
    audioSize=audio.size
    rem=audioSize%16000
    audio=np.append(audio,[np.int16(0)]*(16000-rem))
    path+="_temp.wav"
    scipy.io.wavfile.write(path,sampleRate,audio)
    return path

In [806]:
# All the audio samples are single-channel(mono)
# Loads audio file from path and returns tuple of (sample rate, audio array)
def load_audio(path):
    path=remake_audio(path)
    sampleRate, audio = scipy.io.wavfile.read(path)
    return sampleRate,audio,path

In [807]:
# Time vs. Amplitude plot o original audio, loaded initially.
def plot_audio_init(audio):
    plt.figure(figsize=(12,5))
    plt.plot(audio)
    plt.xlabel("Time")
    plt.ylabel("Amplitude")
    plt.title("Audio Signal")
    plt.show()
    plt.close()

In [ ]:
# Voice-Activity Detection using Silero-VAD.
# Detects first instance of speech in an audio.
# If an error occurs or speech is not detected, function returns tuple of (-1,audio). 
# A check on returned sampleRate can be applied after function implementation to check if speech is detected without error, or not.

def vad(sampleRate,audio,path):
    if sampleRate%16000!=0:
        sampleRate=16000
    wav = read_audio(path,sampling_rate=sampleRate)
    os.remove(path)
    model = load_silero_vad()
    try:
        speech_timestamps = get_speech_timestamps(wav,model,sampling_rate=sampleRate)
    except:
        print(path,sampleRate)
    try:
        x=speech_timestamps[0]['start']
        y=speech_timestamps[0]['end']
        wav=wav[x:y]
        audio = wav.detach().cpu().numpy()
        sampleRate=len(audio)
        return sampleRate,audio
    except:
        print(path)
        return -1,audio

#Time vs Amplitude plot of audio after voice-activity isolation
def plot_audio_vad(audio):
    plt.figure(figsize=(12,5))
    plt.plot(audio)
    plt.xlabel("Time")
    plt.ylabel("Amplitude")
    plt.title("Audio Signal after VAD")
    plt.show()
    plt.close()

In [809]:
# To generate MFCC, we follow the following steps:
# audioInput -> pre-emphasis -> framing -> windowing -> fourier transform -> Inverse Mel Scale Filter Bank -> Log() -> DCT

In [810]:
# Pre-emphasis layer
# Amplifies higher frequencies in order to balance the spectrum (higher frequencies have lower energies)
def pre_emphasize(sampleRate,audio):
    pre_emphasis = 0.97
    audio_preemphasized=[]
    for i in range(1,sampleRate):
        audio_preemphasized=np.append(audio_preemphasized,audio[i]-(audio[i-1]*pre_emphasis))

    return audio_preemphasized

# Time vs. Amplitude plot of audio after pre-emphasis
def plot_audio_pre_emphasis(audio_preemphasized):
    # Plot the pre-emphasized signal
    plt.figure(figsize=(14, 5))
    plt.plot(audio_preemphasized)
    plt.title('Pre-emphasized Signal')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.show()
    plt.close()

In [811]:
# Framing Layer
# Since the audio wave is more than a second, windowing is necesarry in order to fully capture the features and allow for correct
# calculations to be performed. Thus, for ease of calculations, we slice the wave.
# The signal/wave is separated into sections or frames of 25-30 milliseconds.
# Since some parts of the signal are always at the ends of the frames, and we have to perform hamming window, this may result in data loss.
# To tackle this, we frame-shift with a stride of 15ms. This ensures that parts of signals get to be in the center of the signal.

def frame_audio(sampleRate,audio_preemphasized):
    shift_stride=220  # ~10 millisecond of stride
    frame_size=650 # ~30 millisecond frame
    audio_frames=[]

    # Produces 65 audio frames
    for i in range(0,sampleRate-frame_size,shift_stride):
        audio_frames.append(audio_preemphasized[i:i+frame_size])

    return frame_size,audio_frames

# Time vs. Amplitude plot of audio after framing
def plot_audio_frame(audio_frames):
    plt.figure(figsize=(12,4))
    plt.plot(audio_frames[2])
    plt.title('Framed Signal')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.show()
    plt.close()

In [812]:
# Windowing Layer
# Since sudden increase/decrease of amplitude at the edges of the frames create noisy outcomes, we have to smoothen it.
# Thus, we apply hamming window

def hamm_audio(sampleRate,frame_size,audio_frames):
    hammed_audio=[]
    for frame in audio_frames:
        temp_hammed_audio=[]
        for i in range(0,frame_size):
            temp_hammed_audio.append(frame[i]*(0.54-0.46*math.cos(2*math.pi*i/(frame_size-1))))
        
        hammed_audio.append(temp_hammed_audio)

    return hammed_audio

# Time vs. Amplitude plot of audio after application of hamming window
def plot_audio_hammed(hammed_audio):
    plt.figure(figsize=(12,4))
    plt.plot(hammed_audio[2])
    plt.title('Windowed Signal')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.show()
    plt.close()

In [813]:
# FFT(Fast Fourier Transform) Layer
# Used to convert time-domain signal to frequency-domain to analyze frequency components of speech.
# Output of FFT gives complex frequency spectrum (both magnitude and phase)
# Since we only need magnitude, we evaluate the power spectrum from the output of FFT
# NFFT specifies number of points for the FFT. The output is NFFT/2 points

def pow_spec(hammed_audio):
    NFFT=2048
    complex_power_spectrums=np.fft.rfft(hammed_audio,NFFT)
    power_spectrum=(1/NFFT)*pow(np.abs(complex_power_spectrums),2)
    return NFFT,power_spectrum

# Frequency vs. Power/Frequency plot of audio after FFT
def plot_power_spectrum(power_spectrum):
    plt.figure(figsize=(12,4))
    plt.plot(power_spectrum[2])
    plt.title("Power Spectral Density")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Power/Frequency (dB/Hz)")
    plt.show()
    plt.close()

In [814]:
# Mel-filter banks
# Mel-scale related to human-percieved frequency to its actual frequency. Since humans do not hear sound linearly,
# i.e, linear gaps in frequency does not amount to linear change in pitch, we use mel-scale.
# Mel-scale is a logarithm scale, which imitates hearing of humans. Thus, it enables us to capture features as if heard by human.

# Computing the Mel-Filter bank
# 1. Decide upper and lower frequencies in Hertz(SampleRate/2 and 300Hz repectively) 
# 2. Convert them to mels.
# 3. Compute 12 linearly-spaced frequencies inclusive of lower and upper mels.
# 4. Convert these points back to Hertz.
# 5. Round the frequencies to their nearest FFT Bins.
# 6. Create Filterbanks


# Creates frequency bins and returns tuple of (number of filters, frequency bins)
def mels(sampleRate,NFFT):
    freq_to_mel=lambda freq:1125*math.log(1+freq/700)
    lower_hz=300
    upper_hz=sampleRate/2

    lower_mel=freq_to_mel(lower_hz)
    upper_mel=freq_to_mel(upper_hz)

    n_filters=40
    mel_arr=np.linspace(lower_mel,upper_mel,n_filters+2)
    hz_arr=[700*(math.exp((i/1125))-1) for i in mel_arr]

    freq_bin=[math.floor((NFFT+1)*hz_arr_i/sampleRate) for hz_arr_i in hz_arr]
    return n_filters,freq_bin


In [815]:
# Computing the filterbanks.
# Returns filter_banks
def mel_filterbanks(NFFT,n_filters,freq_bin,power_spectrum):
    temp_filter_bank=np.zeros((n_filters,int((NFFT/2))+1))
    for i in range(1,n_filters+1):
        for k in range(0,int((NFFT/2))):  #frame length
            if k<freq_bin[i]:
                temp_filter_bank[i-1][k]=0
            elif freq_bin[i-1]<=k and k<=freq_bin[i]:
                temp_filter_bank[i-1][k]=(k-freq_bin[i-1])/(freq_bin[i]-freq_bin[i-1])
            elif freq_bin[i]<=k and k<=freq_bin[i+1]:
                temp_filter_bank[i-1][k]=(freq_bin[i+1]-k)/(freq_bin[i+1]-freq_bin[i])
            else:
                temp_filter_bank[i-1][k]=0


    filter_banks=np.dot(power_spectrum, temp_filter_bank.T)
    filter_banks=np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)
    filter_banks=np.log(filter_banks+1e-8)

    return filter_banks

# Mel-Spectogram plot of filter banks
def plot_mel_spectogram(sampleRate,filter_banks):
    plt.figure(figsize=(12, 4))
    librosa.display.specshow(filter_banks.T, sr=sampleRate, x_axis='time', y_axis='mel',cmap='turbo')
    plt.colorbar()
    plt.title("Mel-Spectogram")
    plt.tight_layout()
    plt.show()
    plt.close()

In [816]:
# Generate MFCCs
# We apply DCT on the filterbanks to obtain a set of 26 Mel-Frequency Cepstral Coefficients.
# We only require first 13 coefficients for ASR purposes. Rest are to be discarded.
# Returns mfcc
def gen_mfcc(filter_banks):
    mfcc = dct(filter_banks, type=2, axis=1)[:, 1:13] # Keep 2-13
    return mfcc

# Plot of first 13 Mel-Frequency Cepstral Coefficients
def plot_mfcc(sampleRate,mfcc):
    plt.figure(figsize=(12, 4))
    librosa.display.specshow(mfcc.T, sr=sampleRate, x_axis='time', y_axis='mel',cmap='turbo',vmin=-100,vmax=100)
    plt.colorbar()
    plt.title("Mel-Cepstral Frequency Coefficients")
    plt.tight_layout()
    plt.show()
    plt.close()

In [817]:
# To simplify the process of getting MFCC:
#   get_mfcc(path), returns tuple (flag,mfcc).
#       input(String: path)
#       gets audio file from the path and applies necessary functions to obtain Mel-Frequency Cepstral Coefficients.
#       If, during loading of audio or application of VAD, some error occurs, sampleRate

def get_mfcc(path):
    flag=0
    sampleRate,audio,path=load_audio(path)
    sampleRate,audio=vad(sampleRate,audio,path)
    if sampleRate==-1:
        flag=1
        return flag,flag
    audio_preemphasized=pre_emphasize(sampleRate,audio)
    frame_size,audio_frames=frame_audio(sampleRate,audio_preemphasized)
    hammed_audio=hamm_audio(sampleRate,frame_size,audio_frames)
    NFFT,power_spectrum=pow_spec(hammed_audio)
    n_filters,freq_bin=mels(sampleRate,NFFT)
    filter_banks=mel_filterbanks(NFFT,n_filters,freq_bin,power_spectrum)
    mfcc=gen_mfcc(filter_banks)
    return flag,mfcc

def get_mel_filterbanks(path):
    sampleRate,audio,path=load_audio(path)
    sampleRate,audio=vad(sampleRate,audio,path)
    audio_preemphasized=pre_emphasize(sampleRate,audio)
    frame_size,audio_frames=frame_audio(sampleRate,audio_preemphasized)
    hammed_audio=hamm_audio(sampleRate,frame_size,audio_frames)
    NFFT,power_spectrum=pow_spec(hammed_audio)
    n_filters,freq_bin=mels(sampleRate,NFFT)
    filter_banks=mel_filterbanks(NFFT,n_filters,freq_bin,power_spectrum)
    return filter_banks

In [818]:
# Extracts MFCCs from the audio files provided in the input_path and stores the MFCC images at output_path

def prepare_data(input_path,output_path):
    os.makedirs(output_path, exist_ok=True)
    file_names = os.listdir(input_path)
    for file in file_names:
        flag,mfcc=get_mfcc(input_path+file)
        if flag==1:
            continue
        fig=plt.figure(figsize=(1, 1),frameon=False)
        librosa.display.specshow(mfcc.T, x_axis='time', y_axis='mel',cmap='turbo',vmin=-100,vmax=100)   
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(output_path+file+".png")
        plt.close()

In [ ]:
# THIS CELL MUST ONLY BE EXECUTED IF YOU WANT TO EXTRACT MFCCs.
# IF THE MFCCs ARE ALREADY STORED, DO NOT RUN THIS CELL. COMMENT THE CODE BELOW

#For audio containing word "dog"
input_path="audioData/Animals/dog/"
output_path="mfccs_all/dog/"

prepare_data(input_path,output_path)

audioData/Animals/dog/3f2b358d_nohash_0.wav_temp.wav
audioData/Animals/dog/5b09db89_nohash_0.wav_temp.wav
audioData/Animals/dog/72198b96_nohash_1.wav_temp.wav
audioData/Animals/dog/72d75d96_nohash_1.wav_temp.wav
audioData/Animals/dog/84999496_nohash_0.wav_temp.wav
audioData/Animals/dog/9ff2d2f4_nohash_0.wav_temp.wav
audioData/Animals/dog/acfd3bc3_nohash_0.wav_temp.wav
audioData/Animals/dog/d312f481_nohash_1.wav_temp.wav
audioData/Animals/dog/e96a5020_nohash_0.wav_temp.wav
audioData/Animals/dog/fc28c8d8_nohash_0.wav_temp.wav


In [ ]:
# THIS CELL MUST ONLY BE EXECUTED IF YOU WANT TO EXTRACT MFCCs.
# IF THE MFCCs ARE ALREADY STORED, DO NOT RUN THIS CELL. COMMENT THE CODE BELOW

#For audio containing word "cat"
input_path="audioData/Animals/cat/"
output_path="mfccs_all/cat/"

prepare_data(input_path,output_path)

audioData/Animals/cat/00f0204f_nohash_2.wav_temp.wav
audioData/Animals/cat/095847e4_nohash_0.wav_temp.wav
audioData/Animals/cat/0e5193e6_nohash_0.wav_temp.wav
audioData/Animals/cat/16db1582_nohash_1.wav_temp.wav
audioData/Animals/cat/1bc45db9_nohash_0.wav_temp.wav
audioData/Animals/cat/1bc45db9_nohash_1.wav_temp.wav
audioData/Animals/cat/1ffd513b_nohash_1.wav_temp.wav
audioData/Animals/cat/22aa3665_nohash_0.wav_temp.wav
audioData/Animals/cat/32561e9e_nohash_0.wav_temp.wav
audioData/Animals/cat/380abbad_nohash_0.wav_temp.wav
audioData/Animals/cat/39a6b995_nohash_0.wav_temp.wav
audioData/Animals/cat/3ac2e76f_nohash_0.wav_temp.wav
audioData/Animals/cat/4ca37738_nohash_0.wav_temp.wav
audioData/Animals/cat/6205088b_nohash_0.wav_temp.wav
audioData/Animals/cat/62ef962d_nohash_0.wav_temp.wav
audioData/Animals/cat/65d14087_nohash_0.wav_temp.wav
audioData/Animals/cat/66cff190_nohash_0.wav_temp.wav
audioData/Animals/cat/6794a793_nohash_0.wav_temp.wav
audioData/Animals/cat/784e281a_nohash_0.wav_te

In [ ]:
# THIS CELL MUST ONLY BE EXECUTED IF YOU WANT TO EXTRACT MFCCs.
# IF THE MFCCs ARE ALREADY STORED, DO NOT RUN THIS CELL. COMMENT THE CODE BELOW

#For audio containing word "bird"
input_path="audioData/Animals/bird/"
output_path="mfccs_all/bird/"

prepare_data(input_path,output_path)

audioData/Animals/bird/1bc45db9_nohash_0.wav_temp.wav
audioData/Animals/bird/1bc45db9_nohash_1.wav_temp.wav
audioData/Animals/bird/270bfa52_nohash_0.wav_temp.wav
audioData/Animals/bird/31270cb2_nohash_0.wav_temp.wav
audioData/Animals/bird/43691f67_nohash_4.wav_temp.wav
audioData/Animals/bird/44fb4274_nohash_0.wav_temp.wav
audioData/Animals/bird/750e3e75_nohash_0.wav_temp.wav
audioData/Animals/bird/7e783e3f_nohash_0.wav_temp.wav
audioData/Animals/bird/8549f25d_nohash_0.wav_temp.wav
audioData/Animals/bird/9f869f70_nohash_1.wav_temp.wav
audioData/Animals/bird/a8cf01bc_nohash_0.wav_temp.wav
audioData/Animals/bird/b6091c84_nohash_0.wav_temp.wav
audioData/Animals/bird/b7a0754f_nohash_0.wav_temp.wav
audioData/Animals/bird/cdbd6969_nohash_1.wav_temp.wav
audioData/Animals/bird/d9b50b8b_nohash_2.wav_temp.wav
audioData/Animals/bird/d9d6559e_nohash_0.wav_temp.wav
audioData/Animals/bird/e0322f2c_nohash_0.wav_temp.wav
audioData/Animals/bird/e5dadd24_nohash_0.wav_temp.wav
audioData/Animals/bird/e6525

In [ ]:
# Now that the MFCCs are extracted fromt he give audio files, we choose a neural network for classification.
# For current usage and for the sake of experiment and learning, we use a Convolutional Neural Network that takes in whole MFCC
# plot-image of the image and produces a classification based on them.

# However, this method may not be useful when we have an audio file that consists of sentences and not just a single word.
# We may need a recurrent neural network for those purposes.

In [ ]:
# # An MFCC plot-image of an audio is of dimensions (100,100,3)
# # To pass on the images to a CNN, we create a tensor object from the images and use those tensors to train the CNN model.


# # image_dataset_from_directory checks for subdirectories in the path provided, and treats those subdirectories as categories or classes.
# # The images inside those subdirectories are used as data of those categories and are turned into tensors.
# # validation_split=0.2, which means that 20% of all the data is used as validation data, rest is used as training data.
# # subset='training', means that the dataset is to be used for training
# # seed=123, seed for shuffling the order
# # shuffle=True, to shuffle all the tensors
# # labels='inferred', labels are taken from the subdirectories of the path directory.

# # Make sure that MFCCs are extracted before running this cell.
train_dataset=image_dataset_from_directory('mfccs_all/',validation_split=0.2,subset="training",
  seed=123,labels='inferred',image_size=(100,100),batch_size=1,shuffle=True)

# subset='validation', means that the dataset is to be used for validation
validate_dataset=image_dataset_from_directory('mfccs_all/',validation_split=0.2,subset="validation",
  seed=123,labels='inferred',image_size=(100,100),batch_size=1,shuffle=True)

Found 551 files belonging to 3 classes.
Using 441 files for training.
Found 551 files belonging to 3 classes.
Using 110 files for validation.


In [ ]:
# Check the first 3 of data.
# image tensor(tf.tensor) needs to be converted to numpy array and normalized to range 0-1, for appropriate plot.
# label tensor's 0th index contains the label.
# 0 = Bird
# 1 = Cat
# 2 = Dog
# num is used to iterate over num images and labels in dataset
def check_dataset(dataset,num):
    for image_tensor,label_tensor in train_dataset:
        if num<=0:
            break
        image=image_tensor.numpy()
        image=image[0]/255
        label=label_tensor[0].numpy()
        cv2.imshow(f"{label}",image)
        print(label)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        num-=1

    
check_dataset(train_dataset,3)

2
2
0


In [ ]:
# Configuring Convolutional Neural Network to parse and classify MFCC-plots.
# Initial Dimensions of an image are (100,100,3)
model=Sequential()

#Input: (x_dim,y_dim,num_images)
#Input: 100, 100, 1
model.add(Conv2D(8,(3,3),activation='relu',input_shape=(100,100,3),padding='valid'))
model.add(MaxPool2D((2,2),padding='valid'))

#Input: 49, 49, 8
model.add(Conv2D(16,(3,3),activation='relu',padding='valid'))
model.add(MaxPool2D((2,2),padding='valid'))

#Input: 23, 23, 16
model.add(Conv2D(32,(3,3),activation='relu',padding='valid'))
model.add(MaxPool2D((2,2),padding='valid'))

#Input: 10, 10, 32
model.add(Conv2D(64,(3,3),activation='relu',padding='valid'))
model.add(MaxPool2D((2,2),padding='valid'))

#Input: 4, 4, 64
model.add(Flatten())

#Input: 1024 1D-Array
model.add(Dense(1024,activation='relu'))
model.add(Dense(3,activation='softmax'))

In [269]:
model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_36 (Conv2D)              │ (None, 98, 98, 8)      │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_36 (MaxPooling2D) │ (None, 49, 49, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_37 (Conv2D)              │ (None, 47, 47, 16)     │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_37 (MaxPooling2D) │ (None, 23, 23, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_38 (Conv2D)              │ (None, 21, 21, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_38 (MaxPooling2D) │ (None, 10, 10, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_39 (Conv2D)              │ (None, 8, 8, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_39 (MaxPooling2D) │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 3)              │         3,075 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,077,203 (4.11 MB)

 Trainable params: 1,077,203 (4.11 MB)

 Non-trainable params: 0 (0.00 B)

In [270]:
# # Compile multiple models based on prediction values.
# # Here, 7 models have been compiled with different configurations of convolutional layers, number of kernels per layer,
# # and number of fully connected layers with differing neurons.

model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [451]:
# model was trained for 7-8 epochs
history = model.fit(
  train_dataset,
  epochs=1
)

  1/441 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 1.0000 - loss: 8.9165e-05

441/441 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 1.0000 - loss: 8.6704e-05


In [456]:
loss,accuracy=model.evaluate(validate_dataset)

110/110 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9267 - loss: 0.8156


In [ ]:
# # RUN THIS CELL IF YOU WANT TO SAVE YOUR MODEL
# # All the models are saved for later evaluations and comparisons.

# model.save('models/model13.keras')

In [458]:
# Reload the saved models and compare the accuracies over 10 iterations of predictions
# Results are saved in the results dictionary, with keys as mean accuracy and values as model path.

iter=10
model_paths=os.listdir('models/')
results={}
for path in model_paths:
    model=tf.keras.models.load_model('models/'+path)
    total_acc=0
    for i in range(iter):
        loss,accuracy=model.evaluate(validate_dataset,verbose=0)
        total_acc+=accuracy

    results[total_acc/iter]='models/'+path

In [819]:
# Pick the best model
best_model=results[max(results.keys())]
print(f"Best model: {best_model}")
print(f"Avg accuracy: {max(results.keys())}")
model=tf.keras.models.load_model(best_model)
model.save('models/best_model.keras')

Best model: models/model12.keras
Avg accuracy: 0.9545454382896423


In [820]:
# Evaluate model on test images

loss,accuracy=model.evaluate(validate_dataset)

C:\Users\svija\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\backend\tensorflow\nn.py:708: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9686 - loss: 0.2503


In [ ]:
# Now, we predict audio that is not present in the training or validation data.
# These audio files were collected from sources on the internet.

# Extracting MFCCs from the audio files.
prepare_data('test_words/audioData/bird/','test_words/mfccs/bird/')
prepare_data('test_words/audioData/cat/','test_words/mfccs/cat/')
prepare_data('test_words/audioData/dog/','test_words/mfccs/dog/')

48000 57344
torch.Size([64000])
44100 47104
torch.Size([17415])
48000 48128
torch.Size([64000])


In [ ]:
# Make labeled dataset from the MFCCs obtained
test_data=image_dataset_from_directory('test_words/mfccs/',
  seed=123,labels='inferred',image_size=(100,100),batch_size=1)

Found 3 files belonging to 3 classes.


In [ ]:
# Evaluation
loss,accuracy=model.evaluate(test_data)

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 1.0000 - loss: 0.0124

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 0.0104


In [ ]:
# Generate Text from prediction
words={
    0:"Bird",
    1:"Cat",
    2:"Dog"
}
for im,lb in test_data:
    pred=model.predict(im,verbose=3)
    print(f"Prediction: {words[np.argmax(pred)]} \t True Value: {words[lb[0].numpy()]}")